In [1]:
for ii in 1:4
    addprocs(["heathred"])
end

for ii in 1:3
    addprocs(["amon"], dir="")
    addprocs(["zeus"], dir="")
    addprocs(["jove"], dir="")
    addprocs(["ares"], dir="")
end

for ii in 1:10
    #addprocs(["uggp"], dir="")
end

addprocs(6)
workers()

expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error


22-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20
 21
 22
 23

In [2]:
using Iterators
using Pipe
function pz(x :: AbstractArray)
    println(typeof(x), ": ", size(x))
end
macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

In [3]:
@everywhere using RecursiveAutoencoders
@everywhere  using UnfoldingRAE
@everywhere using Zlib

using ClusterSoup

In [4]:
using WordEmbeddings
LL,word_indexes, indexed_words =  load_embeddings("word_emb_data/embeddings-scaled.EMBEDDING_SIZE=50.txt");
size(LL) |> println
word_indexes |> typeof |> println
indexed_words |> typeof |> println

(50,268810)
Dict{String,Int64}
Array{String,1}


In [5]:
training_trees = open("training_sents.jsz","r") do fs
    deserialize(fs)
end;
@assert length(@pipe training_trees |> filter(x->typeof(x)===UTF8String,_))==0
@pz training_trees

training_trees		Array{Any,1}	(3913,)


In [6]:
r_training_trees = r_chunk_data(training_trees)

22-element Array{RemoteRef,1}:
 RemoteRef(2,1,450) 
 RemoteRef(3,1,452) 
 RemoteRef(4,1,454) 
 RemoteRef(5,1,456) 
 RemoteRef(6,1,458) 
 RemoteRef(7,1,460) 
 RemoteRef(8,1,462) 
 RemoteRef(9,1,464) 
 RemoteRef(10,1,466)
 RemoteRef(11,1,468)
 RemoteRef(12,1,470)
 RemoteRef(13,1,472)
 RemoteRef(14,1,474)
 RemoteRef(15,1,476)
 RemoteRef(16,1,478)
 RemoteRef(17,1,480)
 RemoteRef(18,1,482)
 RemoteRef(19,1,484)
 RemoteRef(20,1,486)
 RemoteRef(21,1,488)
 RemoteRef(22,1,490)
 RemoteRef(23,1,492)

In [7]:
@everywhere function unpack!(rae::RAE, θ::Vector)
    W_e_len = length(rae.W_e)
    b_e_len = length(rae.b_e)
    W_d_len = length(rae.W_d)
    b_d_len = length(rae.b_d)
    W_e_shape = size(rae.W_e)
    W_d_shape = size(rae.W_d)
    
    rae.W_e = reshape(θ[1: W_e_len],W_e_shape)
    rae.b_e = θ[W_e_len+1: W_e_len+b_e_len]
    rae.W_d = reshape(θ[W_e_len+b_e_len+1: W_e_len+b_e_len+W_d_len],W_d_shape)
    rae.b_d = θ[W_e_len+b_e_len+W_d_len+1: end]
    
    rae
end

@everywhere function pack(rae::RAE)
    pack(rae.W_e,rae.b_e, rae.W_d,rae.b_d)
end

@everywhere function pack(∇W_e::Matrix{Float64}, ∇b_e::Vector{Float64}, ∇W_d::Matrix{Float64}, ∇b_d::Vector{Float64})
    [∇W_e[:], ∇b_e, ∇W_d[:], ∇b_d] 
end


In [8]:
LL=(LL .- mean(LL,2))./std(LL,2)
LL = LL./(maximum(abs(LL)))
@assert  map(ii->indexed_words[ind2sub(size(LL),ii)[2]], find(abs(LL).>1)) |> unique |> length == 0

In [20]:
function loss!(θ::Vector)  
    #warn("loss! not defined")
    grad = similar(θ)
    #loss_and_loss_grad!(θ::Vector)
    cached_loss_and_loss_grad!(θ, grad)
end

function loss_grad!(θ::Vector, storage::Vector) 
    #warn("loss_grad not defined")
    cached_loss_and_loss_grad!(θ, grad)
end


rae_outer = RAE(LL,word_indexes,indexed_words,0.0001);
#@time r_rae_outers = put!(workers(), rae_outer, 5)
function loss_and_loss_grad!(θ::Vector, grad::Vector)   
    grad[:] = 0
    @inbounds for r_rae_outer in r_rae_outers
        update_remote(r_rae_outer, rae->unpack!(rae, θ) )
    end
    
    function get_remote_loss_grad_function(r_rae_outer::RemoteRef)
        @assert r_rae_outer.where == myid()
        rae = fetch(r_rae_outer)        
        function loss_and_loss_grad(tree::(Any,Any))
            Δs, err = UnfoldingRAE.loss_and_loss_grad(rae, tree)
            [pack(Δs...), err]
        end
    end
    loss_and_loss_grads::Array{RemoteRef,1} = map(r_rae_outers) do r_raeouter
        remotecall(r_raeouter.where, get_remote_loss_grad_function, r_raeouter)::RemoteRef
    end
    
    
    ret = prechunked_mapreduce(r_training_trees, loss_and_loss_grads, (+)) 
    grad[:] = ret[1:end-1]
    err=ret[end]
    
    grad[:]/=length(training_trees)
    err/=length(training_trees)
    err
end

loss_and_loss_grad! (generic function with 1 method)

In [21]:
loss_and_loss_grad_cache = Dict{Vector{Float64},(Float64, Vector{Float64})}()
loss_and_loss_grad_cache_hits = 0
loss_and_loss_grad_cache_misses = 0
function cached_loss_and_loss_grad!(θ::Vector, grad::Vector)
    global loss_and_loss_grad_cache
    global loss_and_loss_grad_cache_hits
    global loss_and_loss_grad_cache_misses
    if haskey(loss_and_loss_grad_cache,θ)
        loss_and_loss_grad_cache_hits+=1
        err, grad[:] = loss_and_loss_grad_cache[θ]
        err
    else
        loss_and_loss_grad_cache_misses+=1
        err = loss_and_loss_grad!(θ, grad)
        loss_and_loss_grad_cache[θ] = (err, grad)
        err
    end
end

cached_loss_and_loss_grad! (generic function with 1 method)

In [ ]:
using NLopt

f_call_count = 0
function tracking_loss_and_loss_grad!(θ::Vector, grad::Vector)
    global f_call_count
    f_call_count+=1
    f_val = cached_loss_and_loss_grad!(θ, grad)   
    println(f_call_count, '\t',f_val,'\t',norm(grad))
    f_val
end
#:LD_MMA, :LD_CCSAQ, :LD_LBFGS, :LD_SLSQP, :LD_VAR2, :LD_VAR1, :LD_TNEWTON_RESTART
opt = Opt(:LD_LBFGS, length(pack(rae_outer)))

#ftol_abs!(opt,1e-9)
maxtime!(opt, 8*60*60)
min_objective!(opt, tracking_loss_and_loss_grad!)

θ = pack(rae_outer)


(optf,optx,ret) = optimize!(opt,θ)


1	18.361596913223025	11.877484135657602
2	61.38673501706334	70.93795059948512
3	14.444538930251985	4.603592735067336
4	13.927956636728727	1.8975807441230113
5	14.444538930251985	1.8975807441230113
6	14.444538930251985	1.8975807441230113
7	13.925939205655235	1.8798380207210357
8	13.907012733403459	1.7053764112630512
9	13.821424056170546	0.8877447088461041
10	13.797971184264858	0.4713005698617467
11	13.821424056170546	0.4713005698617467
12	13.797971184264858	0.4713005698617467
13	13.779281386517948	0.5673194609003832
14	13.821424056170546	0.5673194609003832
15	14.501999192353912	6.275593334975383
16	13.797971184264858	6.275593334975383
17	13.779281386517948	6.275593334975383
18	13.907012733403459	6.275593334975383
19	14.444538930251985	6.275593334975383
20	13.797971184264858	6.275593334975383
21	61.38673501706334	6.275593334975383
22	13.797971184264858	6.275593334975383
23	13.779281386517948	6.275593334975383
24	13.797971184264858	6.275593334975383
25	61.38673501706334	6.275593334975383


In [ ]:
using Optim #https://github.com/JuliaOpt/Optim.jl
f=DifferentiableFunction(loss!,loss_grad!,cached_loss_and_loss_grad!)
#θ = pack(rae_outer)
θ=res.minimum
#res = optimize(f, θ, method=:l_bfgs, show_trace = true, store_trace = true, iterations = 300);
@printval res.f_calls 
@printval res.g_calls 
@printval res.iterations
@printval res.f_minimum
@printval res.gr_converged
@printval res.x_converged 

@printval res.trace
@printval loss_and_loss_grad_cache_hits
@printval loss_and_loss_grad_cache_misses

In [ ]:
res

In [ ]:
θ=pack(rae_outer)
@time loss_and_loss_grad!(θ, zeros(θ))

In [ ]:
Profile.clear()

In [ ]:
tr = ((("Meanwhile", ","), (("staff","are"),("alone","."))))
a = fold(rae_outer,tr)
b= unfold(rae_outer, a)
ĉs = hcat(map(leave-> leave.ĉ, b)...)
bests = show_bests(rae_outer, ĉs)

In [ ]:
bests[:, 1:2:end]